In [1]:
from kafka import KafkaConsumer
from cassandra.cluster import Cluster
from datetime import datetime
import json

#################################################################################################
# OPEN 2 GIT BASH CONSOLES AND RUN ()
# context/kafka/bin/zookeeper-server-start.sh context/kafka/config/zookeeper.properties
# context/kafka/bin/kafka-server-start.sh context/kafka/config/server.properties
#################################################################################################

# Set up Kafka consumer
consumer = KafkaConsumer('candlestickStream',
                        group_id='btcEuro',
                        bootstrap_servers='localhost:9092',
                        enable_auto_commit=True,  # Enable automatic offset commits
                        auto_commit_interval_ms=5000)

cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()
session.set_keyspace('dmsb_tfm')

try:
    # Continuously poll for records and process them
    for message in consumer:
        # Decode and deserialize the message value
        value = json.loads(message.value.decode('utf-8'))
        
        # print(f"Received message: value={value}")
        
        # Check if the start_time exists in the table
        query = f"SELECT count(*) FROM CandleSticks_pred WHERE START_TIME = '{value['start_time']}' ALLOW FILTERING"
        result = session.execute(query).one()[0]
        
        if result == 0: 
            insert_query = session.prepare("""
            INSERT INTO CandleSticks_pred (
                SYMBOL,
                START_TIME,
                OPEN,
                HIGH,
                LOW,
                CLOSE
            )
            VALUES (?, ?, ?, ?, ?, ?)
        """)
            # Execute the insertion query
            # data = [value['start_time'],value['open'],value['high'],value['low'],value['close']]
            session.execute(insert_query, [value['symbol'],datetime.strptime(value['start_time'], '%Y-%m-%d %H:%M:%S'),value['open'],value['high'],value['low'],value['close']])
            # print(f"New row inserted for time: {value['start_time']}")
            # print(data)

except KeyboardInterrupt:
    # Close the consumer connection if the program is terminated by a keyboard interrupt (Ctrl+C)
    consumer.close()
    print('Consumer connection closed')